# Audio to Text

利用Whisper 將語音轉文字
https://huggingface.co/openai/whisper-large-v3

GitHub: https://github.com/openai/whisper




In [ ]:
!pip install torch 
!pip install transformers
!pip install gradio 

In [1]:
## Whisper Speech Recognition Example

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

# AutoModelForSpeechSeq2Seq 是一個來自 Hugging Face 的 Transformers 函式庫中的模型類別。這個模型是用於語音轉文字 (Automatic Speech Recognition, ASR) 的序列到序列 (Seq2Seq) 任務。
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
) 
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# result = pipe(sample)
# print(result["text"])

/Users/linshihuan/Dev/.gradio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.27k/1.27k [00:00<00:00, 1.88MB/s]
model.safetensors: 100%|██████████| 3.09G/3.09G [02:53<00:00, 17.8MB/s]
generation_config.json: 100%|██████████| 3.90k/3.90k [00:00<00:00, 4.97MB/s]
preprocessor_config.json: 100%|██████████| 340/340 [00:00<00:00, 919kB/s]
tokenizer_config.json: 100%|██████████| 283k/283k [00:00<00:00, 469kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.22MB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 4.00MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 10.8MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 266kB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 18.4MB/s]
special_tokens_map.json: 100%|██████

In [2]:
## Speech to text
import gradio as gr

title = "語音 to 文字"

def generateAudio(input):
    print(input)
    result = pipe(input)
    return result["text"]

audio_input = gr.Audio(type="filepath")

app = gr.Interface(fn=generateAudio, inputs=audio_input , outputs="text", title=title)
app.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/private/var/folders/b5/tv_lh5616bgcqzvxbrkkkwdw0000gn/T/gradio/639023028276866d8593ca8df05ecf9d1fa69e52/audio.wav
